<a href="https://colab.research.google.com/github/cisco00/Chat_bot/blob/main/Machine_learning_Projects_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import nltk
import json
import pickle
import numpy as np
import random

from nltk.stem import WordNetLemmatizer, LancasterStemmer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [ ]:
import nltk
nltk.download('all')
nltk.download('punkt')

In [63]:
words = []
classes = []
documents = []
training = []
ignore_words = ['?', '!']
lemmatizer = WordNetLemmatizer()

reading the json files

In [64]:
data_file = open("intents.json").read()
intents = json.loads(data_file)

In [65]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

lemmatizing word and removing duplicate from the list of words and creating a pickle file to store python file objects

In [66]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
#sort classes
classes = sorted(list(set(classes)))
#documents = combination between pattern
print(len(documents), "documents")
#classes = intents
print(len(classes), "classes", classes)
#words = all_words vocabulary
print(len(words), "unique lemmatized words", words)

pickle.dump(words,open("words.pickle", "wb"))
pickle.dump(classes, open("classes.pickle", "wb"))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


creating the training data

In [67]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Building the model

In [68]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

#Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [69]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
10/10 [==============================] - 0s 4ms/step - loss: 2.2165 - accuracy: 0.0851
Epoch 2/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1466 - accuracy: 0.1915
Epoch 3/200
10/10 [==============================] - 0s 4ms/step - loss: 2.0461 - accuracy: 0.2553
Epoch 4/200
10/10 [==============================] - 0s 4ms/step - loss: 1.9869 - accuracy: 0.3191
Epoch 5/200
10/10 [==============================] - 0s 4ms/step - loss: 1.8878 - accuracy: 0.3404
Epoch 6/200
10/10 [==============================] - 0s 4ms/step - loss: 1.7733 - accuracy: 0.3830
Epoch 7/200
10/10 [==============================] - 0s 4ms/step - loss: 1.6568 - accuracy: 0.4894
Epoch 8/200
10/10 [==============================] - 0s 4ms/step - loss: 1.4839 - accuracy: 0.4255
Epoch 9/200
10/10 [==============================] - 0s 4ms/step - loss: 1.2626 - accuracy: 0.7021
Epoch 10/200
10/10 [==============================] - 0s 4ms/step - loss: 1.0988 - accuracy: 0.8085
Epoch 11/

In [70]:
model.save('chatbot_model.h5', hist)
print("model created")

model created


creating a graphic user interface for the chatbot prediction

In [71]:
#loading the model using load_model from tensorflow
from keras.models import load_model
buildt_model = load_model("/content/chatbot_model.h5")

loading files 

In [72]:
intents = json.loads(open("/content/intents.json").read())
words = pickle.load(open("/content/words.pickle", "rb"))
classes = pickle.load(open("/content/classes.pickle", "rb"))

function for cleaning words

In [73]:
def clean_sentences(sentence):
  #tokenizing the words in the sentence
  sentence_word = nltk.wordpunct_tokenize(sentence)
  #create short form of word using stemming
  sentence_words = [lemmatizer.lemmatize(word.lower())for word in sentence_words]
  return sentence_words

In [74]:
def bow(sentence,words, show_details=True):
  #tokenize pattern
  sentence_words = clean_sentences(sentence)

  bag = [0]*len(words)
  for s in sentenece_words:
    for i, w in enumerate(words):
      if w == s:
        bag[i] =1
        if show_details:
          print("found bag of: %" % w)
  return (np.array(bag))

predicting founction

In [75]:
def predict_class(sentence, model):
  #filtering predictions
  p = bow(sentence, words, show_details=False)
  res = model.predict(np.array([p]))[0]
  ERROR_THRESHOLD = 0.25
  results = [[i, r]for i,r in enumerate(res) if r>ERROR_THRESHOLD]

  results.sort(key=lambda x: x[1], reversed=True)

  return_list = []
  for r in results:
    return_list.append({"intents":classes[r[0]], "probability":str(r[1])})
  return return_list  

Responses function

In [76]:
def prediction_response(ints, intents_json):
  tag = ints[0]['intents']
  list_of_intents = intents_json['intents']

  for i in list_of_intents:
    if(i['tag'] == tag):
      result = random.choice(i['response'])
      break
  return result

In [77]:
def chatbot_response(text):
  ints = predict_class(text, model)
  res = prediction_response(ints, intents)

  return res

Creating GUI

In [78]:
import tkinter
from tkinter import *

In [80]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
      ChatLog.config(state=NORMAL)
      ChatLog.insert(END, "you: " + msg + '\n\n')
      ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

      res = chatbot_response(msg)
      ChatLog.insert(END, "Bot: " + res + '\n\n')
      ChatLog.config(state=DISABLED)
      ChatLog.yview(END)

In [ ]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()